In [19]:
pip install textblob nltk


In [20]:
from textblob import TextBlob
import nltk
nltk.download('punkt')
file_path = "/content/Burbank.txt"
with open(file_path, 'r') as file:
    burbank_text = file.read()
blob = TextBlob(burbank_text)
polarity = blob.sentiment.polarity
print("Text Polarity:", polarity)
print("Probability of Negative Sentiment:", negative_probability)


Text Polarity: 0.09869334480780263
Probability of Negative Sentiment: 0.45065332759609866


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
polarity = blob.sentiment.polarity
subjectivity = blob.sentiment.subjectivity
print("Overall Sentiment (Polarity):", polarity)
print("Overall Subjectivity:", subjectivity)


Overall Sentiment (Polarity): 0.09869334480780263
Overall Subjectivity: 0.3790877796901893


In [23]:
from textblob import Word
nltk.download('wordnet')
topics = [Word(word).singularize().lemmatize() for word in blob.words]
unique_topics = set(topics)
print("Key Topics:", unique_topics)

Key Topics: {'path', 'who', 'doing', 'group', 'have', 'putting', 'State', 'head', 'higher', 'added', 'con', 'task', 'Kevin', 'Ramirez', 'each', 'Fernando', 'Next', 'Andre', 'current', 'conduct', 'later', 'relief', 'After', '6:30', 'solve', 'supplying', 'vice', 'little', 'reviewed', 'Karpe', 'Radar', 'Way', 'out', 'agreement', 'immediately', 'did', 'Van', 'study', 'system', 'share', 'airport–basically', 'Californium', 'depending', 'cause', 'FAA', 'became', 'extreme', 'which', 'country', 'implemented', 'implementing', 'found', 'reaching', 'rather', 'recommended', 'discus', 'senior', 'Feb', 'earlier', 'Suzanne', 'meet', 'cheer', 'when', 'news', 'You', 'conducted', 'argue', 'example', 'many', 'change', 'TRACON', 'helpful', 'difficult', 'Wednesday', 'climb', 'Fulton', 'community', 'often', 'at', '2007', 'navigation', 'piece', 'shift', 'term', 'air', 'done', 'Held', 'person', 'UproarLA', 'well', 'operating', 'here', 'worked', 'transportation', 'such', '19', 'dispersing', 'While', 'Administra

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC  # Changed to SVC for simplicity
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from spacy.lang.en.stop_words import STOP_WORDS
import string
df_a = pd.read_csv('/content/amazon_cells_labelled.txt', sep='\t', header=None, names=['review', 'label'])
df_i = pd.read_csv('/content/imdb_labelled.txt', sep='\t', header=None, names=['review', 'label'])
df_y = pd.read_csv('/content/yelp_labelled.txt', sep='\t', header=None, names=['review', 'label'])
df_a['company'] = 'Amazon'
df_i['company'] = 'IMDb'
df_y['company'] = 'Yelp'
data = pd.concat([df_a, df_i, df_y])
data.to_csv('sentiment_data.csv', index=False)
print(data.head())
print("Columns:", data.columns)
print("Null Values:", data.isnull().sum())
nlp = spacy.load('en_core_web_sm')
def tokenize(text):
    return [token.lemma_.lower() for token in nlp(text)
            if token.text not in STOP_WORDS and token.text not in string.punctuation]
class Cleaner(TransformerMixin):
    def transform(self, X, **params):
        return [tokenize(text) for text in X]
    def fit(self, X, y=None, **params):
        return self
    def get_params(self, deep=True):
        return {}
pipeline = Pipeline([
    ('cleaner', Cleaner()),
    ('vectorizer', TfidfVectorizer(tokenizer=lambda txt: txt, lowercase=False)),
    ('classifier', SVC(kernel='linear'))
])

X_train, X_test, y_train, y_test = train_test_split(data['review'], data['label'], test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_test)

for sample, pred in zip(X_test.head(), preds):
    print(sample, "Prediction ➔", 'Positive' if pred == 1 else 'Negative')

print("Accuracy on Training Set:", pipeline.score(X_train, y_train))
print("Accuracy on Test Set:", accuracy_score(y_test, preds))


                                              review  label company
0  So there is no way for me to plug it in here i...      0  Amazon
1                        Good case, Excellent value.      1  Amazon
2                             Great for the jawbone.      1  Amazon
3  Tied to charger for conversations lasting more...      0  Amazon
4                                  The mic is great.      1  Amazon
Columns: Index(['review', 'label', 'company'], dtype='object')
Null Values: review     0
label      0
company    0
dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


It's close to my house, it's low-key, non-fancy, affordable prices, good food. Prediction ➔ Positive
If you stay in Vegas you must get breakfast here at least once. Prediction ➔ Positive
Let's start with all the problemsthe acting, especially from the lead professor, was very, very bad.   Prediction ➔ Negative
It's too bad that everyone else involved didn't share Crowe's level of dedication to quality, for if they did, we'd have a far better film on our hands than this sub-par mess.   Prediction ➔ Negative
i felt insulted and disrespected, how could you talk and judge another human being like that? Prediction ➔ Negative
Accuracy on Training Set: 0.9545040946314831
Accuracy on Test Set: 0.8145454545454546
